In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, LinearRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import scale
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, recall_score, precision_score, plot_precision_recall_curve, mean_squared_error, r2_score
import seaborn as sns
import category_encoders as ce
#import xgboost
#from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
np.random.seed(123)


import xgboost as xgb
pd.options.display.max_columns = None
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn import metrics

Wczytanie danych

In [13]:
df = pd.read_csv("students-all.csv")
# remove rownames
df = df.iloc[:,1:]
df['romantic'] = np.where(df['romantic']=='yes', 1, 0)
df_dummies = pd.get_dummies(df, drop_first=True)
df_dummies.head()

,age,Medu,Fedu,traveltime,studytime,failures,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,school_MS,sex_M,address_U,famsize_LE3,Pstatus_T,Mjob_health,Mjob_other,Mjob_services,Mjob_teacher,Fjob_health,Fjob_other,Fjob_services,Fjob_teacher,reason_home,reason_other,reason_reputation,guardian_mother,guardian_other,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,major_por
0,18,4,4,2,2,0,0,4,3,4,1,1,3,6,5,6,6,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,1,0,0
1,17,1,1,1,2,0,0,5,3,3,1,1,3,4,5,5,6,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0
2,15,1,1,1,2,3,0,4,3,2,2,3,3,10,7,8,10,0,0,1,1,1,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,1,0,1,1,1,0
3,15,4,2,1,3,0,1,3,2,2,1,1,5,2,15,14,15,0,0,1,0,1,1,0,0,0,0,0,1,0,1,0,0,1,0,0,1,1,1,1,1,1,0
4,16,3,3,1,2,0,0,4,3,2,1,2,5,4,6,10,10,0,0,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,1,1,0,0


In [14]:
df_dummies = df_dummies.sample(frac =1)

Zmodyfikowana fun encodująca dane. Beware: target musi być wynikiem encodingu, jeśli romantic znajduje się jako kolumna do zenkodowania -> "romantic_yes"/"romantic_1" itp, dlatego lepiej zamienić "romanitc" ifelsem ręcznie na 0 i 1 dać jako target.

In [15]:

def train_fast(df,target, encoder = None, test_size = None):
    
    df = df.sample(frac = 1)
        
    ret = []
    
    for i in range(5):
        
        df = df.sample(frac = 1)
        X_train, X_test, y_train, y_test = train_test_split(df.drop(target, axis = 1), df[target], test_size = 0.2)
            
        model = xgb.XGBClassifier()
    
    
        if encoder:
            encoder.fit(X_train, y_train)
            X_train = encoder.transform(X_train)

        if encoder: X_test = encoder.transform(X_test)


        model.fit(X_train, y_train)


        y_pred = model.predict_proba(X_test)[:,1]  # positive preds
        
        auc = metrics.roc_auc_score(y_test, y_pred)
        
        ret.append(auc)
    
        
    # returning mean of predictions and model
    return np.array(ret).mean(), model

In [16]:
train_fast(df_dummies, "romantic")

(0.8240050668355494,
 XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
               importance_type='gain', interaction_constraints=None,
               learning_rate=0.300000012, max_delta_step=0, max_depth=6,
               min_child_weight=1, missing=nan, monotone_constraints=None,
               n_estimators=100, n_jobs=0, num_parallel_tree=1,
               objective='binary:logistic', random_state=0, reg_alpha=0,
               reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
               validate_parameters=False, verbosity=None))

In [17]:
def measure_encoder(encoder, model):
    """
    Funkcja działająca jako pipeline w następujących krokach:
    * kodowanie
    * dopasowanie
    * liczenie rmse i r2
    """
    #logreg = LogisticRegression(*args)
    
    X_train, X_test, y_train, y_test = train_test_split(df.drop(['romantic'], axis = 1),df.romantic, test_size = 0.3, random_state = 66)
    
    encoder.fit(X_train, y_train)
    
    X_train = encoder.transform(X_train)
    X_test = encoder.transform(X_test)
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    #mse = mean_squared_error(y_test, y_pred)
    #rmse = np.sqrt(mse)
    #r2 = r2_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    
    return auc

In [18]:
columns= ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'major']

In [19]:
enc = [ce.BackwardDifferenceEncoder(cols=columns),
ce.BaseNEncoder(cols=columns),
ce.BinaryEncoder(cols=columns),
ce.CatBoostEncoder(cols=columns),
ce.HashingEncoder(cols=columns),
ce.HelmertEncoder(cols=columns),
ce.JamesSteinEncoder(cols=columns),
ce.LeaveOneOutEncoder(cols=columns),
ce.MEstimateEncoder(cols=columns),
ce.OneHotEncoder(cols=columns),
ce.OrdinalEncoder(cols=columns),
ce.SumEncoder(cols=columns),
ce.PolynomialEncoder(cols=columns),
ce.TargetEncoder(cols=columns),
ce.WOEEncoder(cols=columns)]

In [20]:
def measure_encodesrs(model):
    res = pd.DataFrame()
    names = ['BackwardDifferenceEncoder',
            'BaseNEncoder',
            'BinaryEncoder',
            'CatBoostEncoder',
             'HashingEncoder',
             'HelmertEncoder',
             'JamesSteinEncoder',
             'LeaveOneOutEncoder',
             'MEstimateEncoder',
             'OneHotEncoder',
             'OrdinalEncoder',
             'SumEncoder',
             'PolynomialEncoder',
             'TargetEncoder',
             'WOEEncoder'
            ]
    for i in range(len(enc)):
        auc = measure_encoder(enc[i], model)
        res = res.append({"Name": names[i], "auc": auc}, ignore_index = True)
    return res

In [21]:
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier, XGBRFClassifier
models = [AdaBoostClassifier(), 
          BaggingClassifier(), 
          ExtraTreesClassifier(),
          RandomForestClassifier(), 
          GradientBoostingClassifier(),
          XGBClassifier(),
          XGBRFClassifier()]

In [22]:
r = []
for m in models:
    print(m)
    print(measure_encodesrs(m))

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=None)
                         Name       auc
0   BackwardDifferenceEncoder  0.547339
1                BaseNEncoder  0.554279
2               BinaryEncoder  0.554279
3             CatBoostEncoder  0.612314
4              HashingEncoder  0.508089
5              HelmertEncoder  0.589993
6           JamesSteinEncoder  0.612314
7          LeaveOneOutEncoder  0.612314
8            MEstimateEncoder  0.612314
9               OneHotEncoder  0.588004
10             OrdinalEncoder  0.547339
11                 SumEncoder  0.588004
12          PolynomialEncoder  0.560732
13              TargetEncoder  0.612314
14                 WOEEncoder  0.612314
BaggingClassifier(base_estimator=None, bootstrap=True, bootstrap_features=False,
                  max_features=1.0, max_samples=1.0, n_estimators=10,
                  n_jobs=None, oob_score=False, random_state=None, verbose

In [145]:
auc_res = []
for e in enc:
    auc,_, = train_fast(df, 'romantic', e)
    auc_res.append(auc)

In [147]:
pd.DataFrame({"encoder": ['BackwardDifferenceEncoder',
            'BaseNEncoder',
            'BinaryEncoder',
            'CatBoostEncoder',
             'HashingEncoder',
             'HelmertEncoder',
             'JamesSteinEncoder',
             'LeaveOneOutEncoder',
             'MEstimateEncoder',
             'OneHotEncoder',
             'OrdinalEncoder',
             'SumEncoder',
             'PolynomialEncoder',
             'TargetEncoder',
             'WOEEncoder'], 
             "auc": auc_res})

,encoder,auc
0,BackwardDifferenceEncoder,0.790684
1,BaseNEncoder,0.845012
2,BinaryEncoder,0.825077
3,CatBoostEncoder,0.821118
4,HashingEncoder,0.629896
5,HelmertEncoder,0.840093
6,JamesSteinEncoder,0.827778
7,LeaveOneOutEncoder,0.820115
8,MEstimateEncoder,0.809501
9,OneHotEncoder,0.812437
